## Welcome Commander!

This notebook will give you an introduction to the Kore Fleet rules and environment.

In [ ]:
%%capture
# install the latest version of kaggle_environments
!pip install --upgrade kaggle_environments

In [ ]:
from kaggle_environments import make
env = make("kore_fleets", debug=True)
print(env.name, env.version)

## Let's start with the shipyard

The shipyard it the only unit you assign actions to. You can either 

1. build more ships
2. launch a fleet of ships

but for now let's just see what a shipyard looks like

In [ ]:
%%writefile do_nothing.py
# First we will make a do_nothing player to observe the game board
def do_nothing():
    pass

In [ ]:
env.run(["/kaggle/working/do_nothing.py"])
env.render(mode="ipython", width=1000, height=800)

# It did nothing!
Your shipyard (in blue) had 0 ships the entire game (denoted by the 0 in the upper left hand corner) and did nothing.

Notice a few things did happen!

1. the kore amount on the tiles with kore grew (the stars got bigger)
2. the number in the bottom righthand tile grew slowly to 10

This number represents the number of ships that can be built a turn at that shipyard! This number is reset to 0 every time a shipyard changes hands!

## Building some ships

Now let's make a simple player that tries to build a ship every turn!

Note that since ships cost 10 `kore` to build and we start with `500`, we will expect to build ships for 50 turns!

In [ ]:
%%writefile builder.py
# this one builds ships!

from kaggle_environments.envs.kore_fleets.helpers import *

def agent(obs, config):
    board = Board(obs, config)
    me=board.current_player

    me = board.current_player
    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    kore_left = me.kore
    
    # loop through all shipyards you control
    for shipyard in me.shipyards:
        # build a ship!
        if kore_left >= spawn_cost:
            action = ShipyardAction.spawn_ships(1)
            shipyard.next_action = action

    return me.next_actions

In [ ]:
env.run(["/kaggle/working/builder.py"])
env.render(mode="ipython", width=1000, height=800)

# We built 50 ships!

You saw the number in the upper left of the shipyard increment to 50 and stop when we ran out of kore! Note that the ui also shows the blue player also no contorls 50 ships.

# To the stars!

Next lets try launching a fleet. A fleet is launched with a series of instructions (go North, South, East, West and follows those, moving every turn). For now let's now worry about that and just launch a fleet to the north.

We expect this fleet to pick up some kore along the way, enabling us to build more ships!

In [ ]:
%%writefile launcher.py
   
from kaggle_environments.envs.kore_fleets.helpers import *

def agent(obs, config):
    board = Board(obs, config)
    me=board.current_player

    me = board.current_player
    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    kore_left = me.kore

    for shipyard in me.shipyards:
        if kore_left >= spawn_cost:
            action = ShipyardAction.spawn_ships(1)
            shipyard.next_action = action
        elif shipyard.ship_count > 0:
            direction = Direction.NORTH
            action = ShipyardAction.launch_fleet_with_flight_plan(2, direction.to_char())
            shipyard.next_action = action

    return me.next_actions

In [ ]:
env.run(["/kaggle/working/launcher.py"])
env.render(mode="ipython", width=1000, height=800)

# Flight Control

Now you might notice a certain weakness of our strategy, it doesn't mine out any of the board! Now we have to make a flight plan which allows our fleet go where no fleet has ever gone before!

In [ ]:
from random import randint
randint(2, 9)

In [ ]:
%%writefile pilot.py
   
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint

# a flight plan
def build_flight_plan(dir_idx, size):
    flight_plan = ""
    for i in range(4):
        flight_plan += Direction.from_index((dir_idx + i) % 4).to_char()
        if not i == 3:
            flight_plan += str(size)
    return flight_plan

def agent(obs, config):
    board = Board(obs, config)
    me=board.current_player

    me = board.current_player
    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    kore_left = me.kore

    for shipyard in me.shipyards:
        if shipyard.ship_count >= 50:
            flight_plan = build_flight_plan(randint(0, 3), randint(2, 9))
            action = ShipyardAction.launch_fleet_with_flight_plan(50, flight_plan)
            shipyard.next_action = action
        elif kore_left >= spawn_cost:
            action = ShipyardAction.spawn_ships(1)
            shipyard.next_action = action

    return me.next_actions

In [ ]:
env.run(["/kaggle/working/pilot.py"])
env.render(mode="ipython", width=1000, height=800)

Notice that if you pause the UI on turn 50, you can see the flight plan your fleet will launch with.

## How flight plans work

1. if it's a direction NESW, change the fleet direction to match
2. if it's a number, decrement the number

examples:

#### N2S (go north, then continue for 2 squares, then go south)
```
N2S
2S
1S
S
(fleet will continue south)
```

#### N10E (go north, then continue for 10 squares, then go east)
```
N10E
10E
9E
8E
7E
6E
5E
4E
3E
2E
1E
E
(fleet will continue east)
```

### Next

In [part 2](https://www.kaggle.com/bovard/kore-intro-ii-mining-kore) we'll go in depth into mining kore!